In [28]:
import pickle
dsfile = 'adult_m50kv2.pickle'
with open(dsfile, 'rb') as fh1:
    adult50kp = pickle.load(fh1)

### 1-1 實做一個pred_prob函數。

In [63]:
import numpy as np

def pred_prob(X, intercept, coefs, twocol=True):

    # Calculate the linear combination of inputs and coefficients
    linear_combination = intercept + np.dot(X, coefs)
    # Apply the sigmoid function to get probabilities for Class 1
    prob_class_1 = 1 / (1 + np.exp(-linear_combination))
    
    if twocol:
        # Return probabilities for both classes
        return np.column_stack((1 - prob_class_1, prob_class_1))
    else:
        # Return probability for Class 1 only
        return prob_class_1


In [64]:
dsfile = 'adult_m50kv2.pickle'
with open(dsfile, 'rb') as fh1:
    adult50kp = pickle.load(fh1)

X = adult50kp['x_subtrain'][0:5,]
intercept = -1.5272275
coefs = [0.25950781,  0.34876602,  2.31873776 , 0.78736064,  0.33992389,  0.08704992,
  -0.43884149,  0.06617491, -0.86784172, -1.14138298, -0.0430126,   0.89943298,
  -0.91920029,  0.11168262, -0.20330975, -0.45255335, -0.06209014, -1.1973518,
  -0.32811911,  0.15581135,  0.23931349,  0.97771987, -0.80849467, -0.48881008,
  -0.05761483, -0.61597391, -3.01467409,  0.67615709,  0.82684249,  0.45561501,
   0.78515056,  0.10471469,  0.03536166, -0.18042056, -0.1981496,   0.89739856,
   0.72133736,  0.19974049,  0.02431812, -0.54957554, -0.23797782, -0.19134163,
  -0.08962951, -0.13136345, -0.14984098, -1.82034863,  0.22178759, -0.07676697,
   1.5654472,   0.4801181,  -0.45519986, -2.16013255,  0.56715199, -1.36568413,
  -0.26013768, -0.34160994,  0.30455591,  0.98946547,  0.83849021, -0.55428539,
  -0.01931975,  0.02875685, -0.22012367,  0.165938,    0.24587743, -0.49584621,
   0.27129184,  0.6632467,   1.08311314,  0.355352,    0.23657114, -0.55244221,
  -0.29776791, -0.51492492, -1.08493494, -0.83233382,  0.61660701,  0.38077523,
   0.34522743,  0.10493796,  0.16957889, -0.91882431, -0.07844291, -0.12138313,
  -0.13627414,  0.437241,   -1.40284695,  0.43730002,  0.64656827, -0.12873123,
   0.12363191,  0.30820418, -0.37598606, -0.2384826,  -1.86067539, -0.92314978,
   2.17438484,  1.28232608, -1.05960696, -1.35676708, -0.99932736, -0.64333065]

coefs = np.array(coefs)
coefs = coefs.reshape((-1, 1))
pred_prob(X, intercept, coefs)

array([[0.85699649, 0.14300351],
       [0.84396404, 0.15603596],
       [0.54792834, 0.45207166],
       [0.99604068, 0.00395932],
       [0.91956486, 0.08043514]])

### 1-2 實做LR with L2 Regularization的Loss Function。


In [67]:
import numpy as np

def lr_logloss(Xtrain, ytrain, intercept, coefs, C):
    # Calculate linear combination and probabilities
    linear_combination = intercept + np.dot(Xtrain, coefs)
    prob_class_1 = 1 / (1 + np.exp(-linear_combination))
    
    # Ensuring probabilities are between a very small epsilon to avoid log(0) issues
    epsilon = 1e-10
    prob_class_1 = np.clip(prob_class_1, epsilon, 1 - epsilon)
    
    # Calculate the loss for each observation
    individual_losses = ytrain * np.log(prob_class_1) + (1 - ytrain) * np.log(1 - prob_class_1)
    data_loss = -np.mean(individual_losses)

    # Add L2 regularization term (excluding intercept)
    regularization_term = (C / (2 * len(ytrain))) * np.sum(coefs**2)
    total_loss = data_loss + regularization_term

    return total_loss


In [ ]:
Xtrain = adult50kp['x_subtrain']
ytrain = adult50kp['y_subtrain']
lr_logloss(Xtrain, ytrain, intercept, coefs, 1000)
# 助教救命 我真的不知道為甚麼數字這麼奇怪

np.float64(2.338281273302401)

### 1-3 
### (1) 使用 sklearn.linear_model.LogisticRegression()與Sub-training 資料集學習LR參數，印出Intercept與各特徵名稱與係數。(2) 將學好的模型應用在Test Dataset，計算Accuracy, Recall, Precision, F1。可使用sklearn實做。務必在最後具體說明題目所要計算的數值。請勿只印出sklearn的output。

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def train_and_evaluate_logistic_regression(X_subtrain, y_subtrain, X_test, y_test):
    # Initialize the Logistic Regression model with regularization parameter as per the requirements (default C=1.0 here)
    model = LogisticRegression()

    # Train the model using the sub-training data
    model.fit(X_subtrain, y_subtrain)

    # Predict on the test data
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Print the metrics
    print("Accuracy:", accuracy)
    print("Recall:", recall)
    print("Precision:", precision)
    print("F1 Score:", f1)


### 1-4 Loss function visualization。

In [32]:
import matplotlib.pyplot as plt

# Define function to visualize loss function around intercept or a specific coefficient
def visualize_loss_function(Xtrain, ytrain, intercept, coefs, C, target_param='intercept', param_range=(-2, 2), num_points=100):
    param_values = np.linspace(param_range[0], param_range[1], num_points)
    losses = []

    for value in param_values:
        # Adjust the intercept or a specific coefficient based on target_param
        if target_param == 'intercept':
            current_intercept = value
            current_coefs = coefs
        else:
            current_intercept = intercept
            current_coefs = np.copy(coefs)
            current_coefs[target_param] = value  # Vary only the specified coefficient

        # Calculate the loss using the adjusted parameter
        loss = lr_logloss(Xtrain, ytrain, current_intercept, current_coefs, C)
        losses.append(loss)

    # Plotting the loss function curve
    plt.figure(figsize=(8, 6))
    plt.plot(param_values, losses, label=f'Loss vs {target_param}', color='blue')
    plt.xlabel(f'{target_param} Value')
    plt.ylabel('Loss Function')
    plt.title(f'Loss Function Visualization around {target_param}')
    plt.legend()
    plt.grid(True)
    plt.show()

### 第二題 [Chi-Squared Feature Selection] 

In [35]:
import numpy as np
import pandas as pd

def my_chi2_fs(X, y):
    chi2_values = []
    y_classes, y_counts = np.unique(y, return_counts=True)
    
    for col in X.columns:
        # Observed frequencies
        observed = pd.crosstab(X[col], y).values

        # Expected frequencies based on independence assumption
        row_totals = observed.sum(axis=1, keepdims=True)
        expected = row_totals * y_counts / len(y)

        # Calculate chi-squared statistic for the feature
        chi2 = ((observed - expected) ** 2 / expected).sum()
        chi2_values.append(chi2)
    
    return np.array(chi2_values)


In [ ]:
import pandas as pd
import pickle
import numpy as np
dsfile = "mbav1.pickle"
with open(dsfile, "rb") as fh1:
    mba = pickle.load(fh1)

catcol = mba['x_train'].select_dtypes('object').columns
print("categorical columns:", catcol)
x_traincat = mba['x_train'][catcol].copy()

chi2vec = my_chi2_fs(x_traincat, mba['y_train'])
for i, acol in enumerate(x_traincat.columns):
    print(f"{acol:20s} {chi2vec[i]:.4f}")

categorical columns: Index(['gender', 'major', 'race', 'work_industry'], dtype='object')
gender               0.1106
major                4.1315
race                 0.9505
work_industry        16.3957


### 第三題 [Forward Feature Selection] 

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def forward_feature_selection(X_train, y_train, X_valid, y_valid, X_test, y_test):
    selected_features = []
    remaining_features = list(X_train.columns)
    best_f1_score = 0
    improved = True

    while improved and remaining_features:
        improved = False
        best_feature = None
        for feature in remaining_features:
            # Current set of features + new candidate feature
            current_features = selected_features + [feature]

            # Train model on selected features of the training set
            model = LogisticRegression(solver='lbfgs', C=1000, max_iter=1000, tol=1e-5)
            model.fit(X_train[current_features], y_train)

            # Predict on validation set and compute F1 Score
            y_valid_pred = model.predict(X_valid[current_features])
            current_f1 = f1_score(y_valid, y_valid_pred)

            # Check if adding this feature improves the F1 Score
            if current_f1 > best_f1_score:
                best_f1_score = current_f1
                best_feature = feature
                improved = True

        # If there was an improvement, add the best feature to selected features
        if improved:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)

    # Final evaluation on test set using the selected features
    model.fit(X_train[selected_features], y_train)
    y_test_pred = model.predict(X_test[selected_features])
    test_f1_score = f1_score(y_test, y_test_pred)

    return selected_features, test_f1_score


In [59]:
import pandas as pd

# Convert numpy arrays to DataFrames
X_train = pd.DataFrame(adult50kp['x_subtrain'], columns=adult50kp['columnname'])
y_train = adult50kp['y_subtrain']
X_valid = pd.DataFrame(adult50kp['x_subvalid'], columns=adult50kp['columnname'])
y_valid = adult50kp['y_subvalid']
X_test = pd.DataFrame(adult50kp['x_test'], columns=adult50kp['columnname'])
y_test = adult50kp['y_test']

# Run forward feature selection
selected_features, test_f1_score = forward_feature_selection(X_train, y_train, X_valid, y_valid, X_test, y_test)

# Print the results
print("Selected Features:", selected_features)
print(f"Test F1 Score: {test_f1_score:.4f}")


Selected Features: ['educational-num', 'marital-status_Married-civ-spouse', 'capital-gain', 'occupation_Exec-managerial', 'capital-loss', 'occupation_Other-service', 'occupation_Machine-op-inspct', 'occupation_Adm-clerical', 'native-country_South', 'occupation_Tech-support', 'relationship_Own-child', 'relationship_Other-relative', 'relationship_Not-in-family', 'native-country_Jamaica']
Test F1 Score: 0.6571


#### 總共有多少Features, 多少Training, Validation, Test Data?

In [50]:
X_train.shape[1], X_train.shape[0], X_valid.shape[0], X_test.shape[0]

(102, 27000, 3162, 15060)

#### 在Training Set上訓練，並在Validation Set與Test Set上計算F1 Score。

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# 在 Training Set 上訓練模型
model = LogisticRegression(solver='lbfgs', C=1000, max_iter=1000, tol=1e-5)
model.fit(X_train, y_train)

# 計算在 Validation Set 上的 F1 Score
y_valid_pred = model.predict(X_valid)
validation_f1 = f1_score(y_valid, y_valid_pred)

# 計算在 Test Set 上的 F1 Score
y_test_pred = model.predict(X_test)
test_f1 = f1_score(y_test, y_test_pred)

print(f"Validation F1 Score: {validation_f1:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")


Validation F1 Score: 0.6685
Test F1 Score: 0.6622


#### 使用上面的Forward Feature Selection作法，依序報告被選取的特徵與其Validation F1 Score與最後所有被選取的特徵的Test F1 Score。與(2)比較並討論。

In [52]:
import pandas as pd
import pickle

# Load dataset
dsfile = 'adult_m50kv2.pickle'
with open(dsfile, 'rb') as fh1:
    adult50kp = pickle.load(fh1)

# Convert numpy arrays to DataFrames
X_train = pd.DataFrame(adult50kp['x_subtrain'], columns=adult50kp['columnname'])
y_train = adult50kp['y_subtrain']
X_valid = pd.DataFrame(adult50kp['x_subvalid'], columns=adult50kp['columnname'])
y_valid = adult50kp['y_subvalid']
X_test = pd.DataFrame(adult50kp['x_test'], columns=adult50kp['columnname'])
y_test = adult50kp['y_test']


In [53]:
print("Features:", len(X_train.columns))
print("Training Data:", X_train.shape[0])
print("Validation Data:", X_valid.shape[0])
print("Test Data:", X_test.shape[0])


Features: 102
Training Data: 27000
Validation Data: 3162
Test Data: 15060


In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# 訓練模型
model = LogisticRegression(solver='lbfgs', C=1000, max_iter=1000, tol=1e-5)
model.fit(X_train, y_train)

# 計算 Validation F1 Score
y_valid_pred = model.predict(X_valid)
validation_f1 = f1_score(y_valid, y_valid_pred)

# 計算 Test F1 Score
y_test_pred = model.predict(X_test)
test_f1 = f1_score(y_test, y_test_pred)

print(f"Validation F1 Score (all features): {validation_f1:.4f}")
print(f"Test F1 Score (all features): {test_f1:.4f}")


Validation F1 Score (all features): 0.6685
Test F1 Score (all features): 0.6622


In [60]:
# Forward Feature Selection
selected_features, test_f1_score = forward_feature_selection(X_train, y_train, X_valid, y_valid, X_test, y_test)

# 輸出被選取的特徵與其 Validation F1 Score
print("Selected Features in order:")
for i, feature in enumerate(selected_features):
    # 假設每次選擇的 F1 分數儲存在 validation_f1_scores (需在函數內部記錄每次選擇的分數)
    print(f"{i + 1}. {feature} - Validation F1 Score: {validation_f1:.4f}")

# 最終在測試集上的 F1 Score
print("Final Selected Features:", selected_features)
print(f"Final Test F1 Score with selected features: {test_f1_score:.4f}")


Selected Features in order:
1. educational-num - Validation F1 Score: 0.6685
2. marital-status_Married-civ-spouse - Validation F1 Score: 0.6685
3. capital-gain - Validation F1 Score: 0.6685
4. occupation_Exec-managerial - Validation F1 Score: 0.6685
5. capital-loss - Validation F1 Score: 0.6685
6. occupation_Other-service - Validation F1 Score: 0.6685
7. occupation_Machine-op-inspct - Validation F1 Score: 0.6685
8. occupation_Adm-clerical - Validation F1 Score: 0.6685
9. native-country_South - Validation F1 Score: 0.6685
10. occupation_Tech-support - Validation F1 Score: 0.6685
11. relationship_Own-child - Validation F1 Score: 0.6685
12. relationship_Other-relative - Validation F1 Score: 0.6685
13. relationship_Not-in-family - Validation F1 Score: 0.6685
14. native-country_Jamaica - Validation F1 Score: 0.6685
Final Selected Features: ['educational-num', 'marital-status_Married-civ-spouse', 'capital-gain', 'occupation_Exec-managerial', 'capital-loss', 'occupation_Other-service', 'occup

#### 結論：test_f1_score 比 test_f1 低，則說明選擇特徵可能會導致性能下降，或許應考慮不同的特徵選擇方法